## Google Net (Inception V1 모델)
- 구글의 체게디 등이 개발 , 2014 우승
- Google Net 과 VGGNet은 동등한 시기에 나왔으나, VGGNet 이 좀 더 간편해서 VGG가 더 많이 유명해짐
- 12x less params than AlexNet

![googlenet](img/googlenet.png)

### GoogleNet 배경
- CNN을 구성할시 고정된 Architecture구조를 사용하는 것보다 **Mobile or Embedded Computing 등 다양한 곳에서 자유롭게 사용될 수 있는 유연한 구조를 갖고 있어야 된다**고 생각

## GoogleNet 구조 및 Architecture
- 전체 22 Layers
- Efficient Inception module
- NO FC Layers

<hr>


## 1x1 연산
- 기존 연산에서는 다 더하면 28x28x672이고 이값은 레이어가 깊어질수록 점점더 커진다.
- **1x1연산은 동일 w, h 여도 차원(depth) 를 감소시켜준다.**
    - 약간 28x28x56 -> 28x28x32 원리가 dot 연산 느낌
- 이후 연산에서는 다 더해도 비교적 작아짐
- 단 1x1연산은 정보 손실있지만, redundancy가 있는 input features를 선형 결합하고, non-linear를 추가하기 때문에 좋은 효과를 낼 수 있음
- 이러한 Inception 모듈을 쌓아나가는 방식
- Maxpooling은 stride가 1임

![googlenet_1](img/googlenet_1.PNG)

![googlenet_2](img/googlenet_2.PNG)

<hr>



## GoogleNet 주요 포인트
- (1) 파라메터 수를 줄이자! = Channel(dimension) Reduction
    - 1x1 conv(width와 height는 줄지 않고 채널별로 묶음)
        - ex) 기존 56x56x64 에서 1x1 conv했더니 56x56x32가 된다는건 커널수로 곱셈된 것. 
    - **차원수 64짜리를 32짜리로 정보 압축 하는 느낌**
    - Tensor factorization (행렬을 다른 행렬로 분해 했을떄 파라미터가 줄수있다)
- (2) 연산을 효율적으로 하려면 어떻게 해야할까?
    - matrix 연산이 dense해야 한다.
- (3) 깊은 layer까지 정보를 전달 (gradient descent)
    - 보조분류기(auxiliary classifier) 
        - 가중치를 쭉 진행시 gradient descent가 발생
        - 중간에 저장(끝내고)하고, 다음것부터 이제 다시 저장한 정답값과 오차 계산해나감
    - **즉 역전파에서 기울기 소실이 발생하는 것을 방지하기 위해 같은 문제를 여러 단계서 풀음**
    - 기울기 소멸 문제 완화 장치로 4번째, 7번째 계층에 보조 분류기 추가 / 보조 분류기를 통해 그레디언트 정보 제공
    - **학습때는 auxiliary classifier 사용, 추론시에는 사용하지 않고 마지막 classifier만 사용**

- (4) Overfitting이 덜되는 general 한 구조 
    - 네트워크를 sparse 한 convolution 으로 진행
- ==> Inception 모듈의 등장

- (5) FC 대신, Global Average Classification 사용


## Inception 모듈
![inception_module1](img/inception_module1.png)
- 다양한 feature를 뽑기 위해 여러 conv 병렬 사용(local sparse structure conv)
- 마지막엔 이를 concatenation 하는데 이 과정이 dense matrix 이다. 
- 1x1 conv / 3x3 conv / 5x5 conv / 3x3 maxpooling

![inception_module2](img/inception_module2.png)
- (a)보다 (b)가 더 성능을 잘낸다.
- **(b) 는 1x1이 다 들어갔는데, 1x1이 들어갔을 때, 확실히 non-linear 특성을 증가시키면서, 채널들간의 linear combination 가능 ==> 1x1 dimension reduction**

- 마지막 Dense 로 모아주고 다시 경우를 나누고 펼치고 다시 모으고 이런식
- **1x1 컨볼루션 : 동일한 위치의 특징맵 값을 필터의 가중치와 선형 결합, 1x1 컨볼루션 필터의 개수를 조정하여 출력되는 특징지도의 개수를 조정**

<hr>

### Inception Module - Basic,  A, B, C 차이
- Module Basic : 5x5, 3x3, 1x1, pooling 사용
- Module A : VGG영향으로 5x5를 3x3 2개로 변경 모듈
- Module B : Tensor Facotrization 두 번 적용 된 것 (NxN -> 1xn, nx1)
- Module C : Grid Reduction 적용 된 것 (Pooling vs Conv)

![inception_module](img/inception_module.png)